In [2]:
import os

In [4]:
# Load dataset from Hugging Face datasets library
from datasets import load_dataset, concatenate_datasets
DATA_DIR =f"/home/pranav-pc/projects/OpenTransformer/multiformer/data"
dataset_train = load_dataset(path=os.path.join(DATA_DIR,'downloads/TinyStories'),trust_remote_code=True,split='train')
dataset_val = load_dataset(path=os.path.join(DATA_DIR,'downloads/TinyStories'),trust_remote_code=True,split='validation')

dataset = concatenate_datasets([dataset_train,dataset_val])
del dataset_train
del dataset_val

In [12]:
from src.tokenize.tokenizer import Tokenizer
TOKENIZER_CHECKPOINT = (
    "/home/pranav-pc/projects/OpenTransformer/multiformer/tokenizer_checkpoints/"
)

tokenizer = Tokenizer(TOKENIZER_CHECKPOINT)


In [13]:
text = dataset[0]['text']
text

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'

In [18]:
print(tokenizer.encode(text,out_type=str))

['<s>', '▁One', '▁day', ',', '▁a', '▁little', '▁girl', '▁named', '▁L', 'ily', '▁found', '▁a', '▁need', 'le', '▁in', '▁her', '▁room', '.', '▁She', '▁knew', '▁it', '▁was', '▁difficult', '▁to', '▁play', '▁with', '▁it', '▁because', '▁it', '▁was', '▁sharp', '.', '▁L', 'ily', '▁wanted', '▁to', '▁share', '▁the', '▁need', 'le', '▁with', '▁her', '▁mom', ',', '▁so', '▁she', '▁could', '▁se', 'w', '▁a', '▁button', '▁on', '▁her', '▁sh', 'irt', '.', '<0x0A>', '<0x0A>', 'L', 'ily', '▁went', '▁to', '▁her', '▁mom', '▁and', '▁said', ',', '▁"', 'M', 'om', ',', '▁I', '▁found', '▁this', '▁need', 'le', '.', '▁Can', '▁you', '▁share', '▁it', '▁with', '▁me', '▁and', '▁se', 'w', '▁my', '▁sh', 'irt', '?"', '▁Her', '▁mom', '▁smiled', '▁and', '▁said', ',', '▁"', 'Yes', ',', '▁L', 'ily', ',', '▁we', '▁can', '▁share', '▁the', '▁need', 'le', '▁and', '▁fix', '▁your', '▁sh', 'irt', '."', '<0x0A>', '<0x0A>', 'T', 'ogether', ',', '▁they', '▁shared', '▁the', '▁need', 'le', '▁and', '▁se', 'wed', '▁the', '▁button', '▁on', '

In [ ]:
tokenizer.

In [ ]:

data = dataset.map(
    lambda example: {"idx": [en[:block_size] for en in tokenizer.encode(example["text"])]},
    batch_size=512,
    batched=True,
    remove_columns=dataset["train"].column_names,
)


# Define collate function to handle padding
def collate_fn(batch):
    x_batch = [torch.tensor(en[:-1]) for en in batch]  # Extract x (remove last token)
    y_batch = [torch.tensor(en[1:]) for en in batch]  # Extract y (remove first token)
    x_padded = pad_sequence(
        x_batch, batch_first=True, padding_value=tokenizer.eos_id()
    )  # Pad x sequences
    y_padded = pad_sequence(
        y_batch, batch_first=True, padding_value=tokenizer.eos_id()
    )  # Pad y sequences
    return x_padded, y_padded


# Sort the data and turn off shuffle - Simplest way of implementing Seq leng batch sampling
train_data = sorted(data["train"]["idx"], key=lambda x: len(x))

# Create DataLoader with collate function
train_loader = DataLoader(
    train_data, batch_size=batch, collate_fn=collate_fn, shuffle=False
)
